In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/toxic-model/toxic.h5
/kaggle/input/saved-model/toxic_rnn.h5
/kaggle/input/glove-vector/glove.6B.100d.txt
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation-processed-seqlen128.csv


In [2]:
from __future__ import print_function
from builtins import range
import tensorflow as tf
from kaggle_datasets import KaggleDatasets
import transformers
from transformers import TFAutoModel, AutoTokenizer
#try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
#    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#    print('Running on TPU ', tpu.master())
#except ValueError:
 #   tpu = None

#if tpu:
 #   tf.config.experimental_connect_to_cluster(tpu)
  #  tf.tpu.experimental.initialize_tpu_system(tpu)
   # strategy = tf.distribute.experimental.TPUStrategy(tpu)
#else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
 #   strategy = tf.distribute.get_strategy()

#print("REPLICAS: ", strategy.num_replicas_in_sync)

#AUTO = tf.data.experimental.AUTOTUNE

# Data access
MODEL = 'PD_TOXIC_X_CLASS'
# Set your own project id here
#PROJECT_ID = 'pd@toxic1234'
#from google.cloud import storage
#storage_client = storage.Client(project=PROJECT_ID)
#from google.cloud import bigquery
#bigquery_client = bigquery.Client(project=PROJECT_ID)

In [3]:
import os,sys,numpy as np,pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM,Bidirectional,Dense,GlobalMaxPooling1D,Embedding,Input,MaxPooling1D
from keras.models import Model
from keras.optimizers import RMSprop
from sklearn.metrics import roc_auc_score
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint

Using TensorFlow backend.


In [4]:
#train data and test labels 
train = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv')
train.head()
#load word vectors
word2vec = {}
with open(os.path.join('/kaggle/input/glove-vector/glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray((values [1:]))
        word2vec[word] = vec
print('total vocab in glove',len(word2vec))
        
        
        
valid = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

y_valid = valid.toxic.values

total vocab in glove 400000


In [5]:
print(train.shape)

(223549, 8)


In [6]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [7]:
train.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
223544,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0
223545,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0
223546,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0
223547,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0
223548,fffb5451268fb5ba,""" \n\n == Unicorn lair discovery == \n\n Suppo...",0,0,0,0,0,0


In [8]:
max_seq_len = 20
max_voc_size = 400000
embed_dim = 100
val_split = 0.2
batch_size = 128
epoch = 5

In [9]:
#extract sentences
sentences = train['comment_text'].fillna('DUMMY_VALUES').values
#target
labels = ['toxic' ,'severe_toxic' ,'obscene' ,'threat' ,'insult' ,'identity_hate']
target  = train[labels].values
print(sentences[1:2])
print(target[1:2])

#SENTENCE FOR TEST
#sentences_test = test['comment_text'].fillna('DUMMY_VALUES').values

#SENTENCE FOR validate
#sentences_validate = valid['comment_text'].fillna('DUMMY_VALUES').values


["D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"]
[[0 0 0 0 0 0]]


In [10]:
tokenizer = Tokenizer(num_words = max_voc_size)
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
#sequences test
#sequences_test = tokenizer.texts_to_sequences(sentences_test)

#sequences validate
#sequences_validate = tokenizer.texts_to_sequences(sentences_validate)

In [11]:
print(len(sequences))

223549


In [12]:
#word2idx
word2idx = tokenizer.word_index

In [13]:
#padding 
padded_seq = pad_sequences(sequences,maxlen=max_seq_len)

#padding test data
#padded_seq_test = pad_sequences(sequences_test , maxlen = max_seq_len)

#padding the validate data
#padded_seq_validate = pad_sequences(sequences_validate , maxlen = max_seq_len)

In [14]:
print(padded_seq.shape)

(223549, 20)


In [15]:
#embedding matrix
num_words = min(max_voc_size,len(word2idx)+1)
embed_matrix = np.zeros((num_words,embed_dim))
for word , i in word2idx.items():
    if i < max_voc_size:
        #getting word vector
        word_vec = word2vec.get(word)
        if word_vec is not None:
            embed_matrix[i]=word_vec

In [16]:
#embedding layer
embed_layer = Embedding(num_words,embed_dim,
                        weights = [embed_matrix],
                       input_length=max_seq_len,
                       trainable = True)


In [17]:
#building model
from keras.optimizers import Adam
from keras.layers import Activation
import tensorflow as tf
# detect and init the TPU
#tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#tf.config.experimental_connect_to_cluster(tpu)
#tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
#tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
#with tpu_strategy.scope():
    
input_ = Input(shape = (20,))
# = embed_layer(input_)
x = Embedding(num_words , 20,input_length=max_seq_len)(input_)
x = LSTM(20,return_sequences=True)(x)
x = MaxPooling1D(3)(x)
x = Bidirectional(LSTM(20,return_sequences = True))(x)
x = GlobalMaxPooling1D()(x)
x = Dense(6)(x)
out = Activation('sigmoid')(x)
model = Model(input_,out)
model.compile(loss = 'binary_crossentropy',
             optimizer = Adam(lr = 0.001),
             metrics = ['categorical_accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 20, 20)            6005160   
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 20)            3280      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 6, 20)             0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 6, 40)             6560      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 40)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 246 

In [18]:
#implementing callbacks
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
checkpoint = ModelCheckpoint("toxic_rnn.h5",
                            monitor="val_loss",
                            mode="min",
                            save_best_only=True,
                            verbose=1)
early_stopping = EarlyStopping(monitor="val_loss",
                              min_delta=0,
                              patience=3,
                              verbose=1,
                              restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss",
                             factor=0.1,
                             patience=3,
                             verbose=1,
                             min_delta=0.0001)
#putting callbacks in callbacks list
callbacks = [checkpoint,early_stopping,reduce_lr]

In [19]:
r = model.fit(padded_seq,target,batch_size=batch_size,epochs=epoch,validation_split = 0.2 ,callbacks=callbacks)

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 178839 samples, validate on 44710 samples
Epoch 1/5
178839/178839 [==============================] - 61s 340us/step - loss: 0.1207 - categorical_accuracy: 0.9726 - val_loss: 0.0925 - val_categorical_accuracy: 0.9969

Epoch 00001: val_loss improved from inf to 0.09254, saving model to toxic_rnn.h5
Epoch 2/5
178839/178839 [==============================] - 56s 312us/step - loss: 0.0707 - categorical_accuracy: 0.9945 - val_loss: 0.0937 - val_categorical_accuracy: 0.9974

Epoch 00002: val_loss did not improve from 0.09254
Epoch 3/5
178839/178839 [==============================] - 55s 306us/step - loss: 0.0629 - categorical_accuracy: 0.9946 - val_loss: 0.1021 - val_categorical_accuracy: 0.9974

Epoch 00003: val_loss did not improve from 0.09254
Epoch 4/5
178839/178839 [==============================] - 55s 305us/step - loss: 0.0571 - categorical_accuracy: 0.9946 - val_loss: 0.1071 - val_categorical_accuracy: 0.9974

Epoch 00004: val_loss did not improve from 0.09254
Restoring model

In [20]:
model.save('/kaggle/working/toxic_softmax3_notglove.h5')

In [21]:
!pip install simplejson

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [22]:
import simplejson as json

In [23]:
tokenizer_json = tokenizer.to_json()
with open('tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

In [24]:
from keras.models import load_model

In [25]:
model2 = load_model('/kaggle/working/toxic_softmax3_notglove.h5')

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [26]:
model2.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 20, 20)            6005160   
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 20)            3280      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 6, 20)             0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 6, 40)             6560      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 40)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 246 

In [27]:
sentences_new = train['comment_text'].fillna('DUMMY_VALUES').values

In [28]:
from keras.preprocessing.text import tokenizer_from_json
with open('tokenizer.json') as f:
    data = json.load(f)
    tokenizer4 = tokenizer_from_json(data)




#tokenizer2 = Tokenizer(num_words = 20000)
#tokenizer2.fit_on_texts(sentences_new)
sequences_cus = tokenizer4.texts_to_sequences(sentences_new)

In [29]:
word2idx = tokenizer2.word_index
print(word2idx[FUCKED])

NameError: name 'tokenizer2' is not defined

In [30]:
#s = ['COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK']
s = ['I amm happy for you brother']
s2 = ['i will kill you motherfucker']

In [31]:
#tokenizer2 = Tokenizer(num_words =  20000)
#tokenizer.fit_on_texts(s)
sequences_custom = tokenizer4.texts_to_sequences(s)
sequences_custom2 = tokenizer4.texts_to_sequences(s2)
print(sequences_custom)
print(sequences_custom2)

[[7, 473, 12, 6, 2144]]
[[7, 44, 795, 6, 2712]]


In [32]:
padded_seq_cus = pad_sequences(sequences_custom,maxlen=20,padding  = 'post')
padded_seq_cus_2 = pad_sequences(sequences_custom2,maxlen=20,padding  = 'post')

In [33]:
print(padded_seq_cus_2)
print(padded_seq_cus_2.shape)
print(padded_seq_cus)
print(padded_seq_cus.shape)

[[   7   44  795    6 2712    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]]
(1, 20)
[[   7  473   12    6 2144    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]]
(1, 20)


In [34]:
x = model2.predict(padded_seq_cus)
x2 = model2.predict(padded_seq_cus_2)

In [35]:
print(x.shape)
print(x2.shape)

(1, 6)
(1, 6)


In [36]:
print(x)
print(x2)

[[0.04101384 0.0014082  0.01564648 0.00086248 0.01461239 0.00354151]]
[[0.8083033  0.09517158 0.6449795  0.04238441 0.54048836 0.11803702]]


In [37]:
def category(arr):
    label = ['toxic','severe','obscene','threat','insult','identity_hate']
    for a in arr:
        for x in range(6):
            print ('sentence is {} percent toxic of category {} '.format(a[x],label[x]))
print(category(x))
print(category(x2))

sentence is 0.041013844311237335 percent toxic of category toxic 
sentence is 0.001408196403644979 percent toxic of category severe 
sentence is 0.01564648374915123 percent toxic of category obscene 
sentence is 0.0008624771144241095 percent toxic of category threat 
sentence is 0.014612388797104359 percent toxic of category insult 
sentence is 0.003541512181982398 percent toxic of category identity_hate 
None
sentence is 0.8083032965660095 percent toxic of category toxic 
sentence is 0.09517157822847366 percent toxic of category severe 
sentence is 0.6449794769287109 percent toxic of category obscene 
sentence is 0.04238441213965416 percent toxic of category threat 
sentence is 0.5404883623123169 percent toxic of category insult 
sentence is 0.11803702265024185 percent toxic of category identity_hate 
None


In [38]:
def loaded(name):
    from keras.preprocessing.text import tokenizer_from_json
    with open(name) as f:
        data = json.load(f)
        tokenizer_loaded = tokenizer_from_json(data)
        return tokenizer_loaded
def load_saved_model(location):
     model_infer = load_model('/kaggle/working/toxic_softmax3_notglove.h5')
     return model_infer

def pipeline(lis):
    name = 'tokenizer.json'
    location = '/kaggle/working/toxic_softmax3_notglove.h5'
    model_infer = load_saved_model(location)
    tokenizer_pipe = loaded(name)
    sequences_custom_pipe = tokenizer_pipe.texts_to_sequences(lis)
    padded_seq_cus_pipe = pad_sequences(sequences_custom_pipe,maxlen=20,padding  = 'post')
    pred = model_infer.predict(padded_seq_cus_pipe)
    category(pred)

In [39]:
lis = ['i want to kill you till death']
lis2 = ['motherfucker fuck you man']
lis3 = ['i am happy that you got the job']
pipeline(lis)
pipeline(lis2)
pipeline(lis3)

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


sentence is 0.5331634283065796 percent toxic of category toxic 
sentence is 0.029372794553637505 percent toxic of category severe 
sentence is 0.28596416115760803 percent toxic of category obscene 
sentence is 0.014876600354909897 percent toxic of category threat 
sentence is 0.26000919938087463 percent toxic of category insult 
sentence is 0.04867182672023773 percent toxic of category identity_hate 


/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


sentence is 0.8284076452255249 percent toxic of category toxic 
sentence is 0.11493822187185287 percent toxic of category severe 
sentence is 0.6836121678352356 percent toxic of category obscene 
sentence is 0.058817800134420395 percent toxic of category threat 
sentence is 0.583493173122406 percent toxic of category insult 
sentence is 0.13760221004486084 percent toxic of category identity_hate 


/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


sentence is 0.04195234552025795 percent toxic of category toxic 
sentence is 0.0014813330490142107 percent toxic of category severe 
sentence is 0.015989087522029877 percent toxic of category obscene 
sentence is 0.0009036391857080162 percent toxic of category threat 
sentence is 0.014486114494502544 percent toxic of category insult 
sentence is 0.003547986736521125 percent toxic of category identity_hate 
